In [ ]:
%%capture
import os
from pathlib import Path
import pandas as pd
from dj_notebook import activate
import numpy as np

env_file = os.environ["META_ENV"]
reports_folder = Path(os.environ["META_REPORTS_FOLDER"])
analysis_folder = Path(os.environ["META_ANALYSIS_FOLDER"])
pharmacy_folder = Path(os.environ["META_PHARMACY_FOLDER"])
plus = activate(dotenv_file=env_file)

In [ ]:
from edc_pharmacy.utils import get_stock_for_location_df
from edc_pharmacy.models import Location, Stock, Dispense, StockRequest
from django.db.models import Count
from edc_pharmacy.utils import get_instock_and_nostock_data
from edc_pharmacy.analytics import get_next_scheduled_visit_for_subjects_df



In [ ]:
location = Location.objects.all()[2]
get_stock_for_location_df(location)

In [ ]:

qs_stock = (
    Stock.objects.values(
        "allocation__registered_subject__subject_identifier",
        "code",
        "dispense__dispense_identifier",
        "location__name",
    )
    .filter(location=location, qty=1)
    .annotate(count=Count("allocation__registered_subject__subject_identifier"))
)


In [ ]:
qs_stock

In [ ]:
Dispense.objects.all()[0].__dict__

In [ ]:
Stock.objects.filter(dispense__isnull=False)

In [ ]:
stock_request = StockRequest.objects.get(request_identifier="000122")
df_next_scheduled_visits = get_next_scheduled_visit_for_subjects_df(stock_request)
df_instock, df_nostock = get_instock_and_nostock_data(stock_request, df_next_scheduled_visits)



In [ ]:
df_instock

In [ ]:
df_nostock

In [ ]:
from edc_pdutils.dataframes import get_appointments

from edc_utils import get_utcnow
from edc_appointment.constants import NEW_APPT
from dateutil.relativedelta import relativedelta
from datetime import datetime

df_appt = get_appointments()

mask1 = (df_appt.visit_code.isin([1240.0, 1360.0, 1480.0]))
mask2 = (df_appt.appt_datetime > datetime(2026, 1, 1)) & (df_appt.appt_datetime < datetime(2026, 8, 1))
df = df_appt[(df_appt.appt_status==NEW_APPT) & mask1 & mask2]
df.subject_identifier.nunique()


In [ ]:

df = df_appt[(df_appt.appt_status!=NEW_APPT) & (df_appt.visit_code.isin([1240.0, 1360.0, 1480.0]))].visit_code.value_counts()
df.sum()

In [ ]:
from tabulate import tabulate
print(tabulate(df.to_frame(), headers="keys"))

In [ ]:

df_appt.appt_datetime < datetime(2025, 12, 1)